https://platform.openai.com/docs/guides/function-calling

--> Open AI에서 설명하는 function calling에 대한 url

#### Function Calling의 사용
- 이 코드에서 function calling은 모델이 적절한 시점에 외부 함수(get_current_weather)를 호출하도록 유도합니다. 모델이 What is the current weather in {user_location}?와 같은 질문에 응답할 때, 해당 함수가 정의되어 있으면 모델은 그 함수를 호출하여 날씨 정보를 가져오려 합니다.

- function_call="auto" 설정을 통해 GPT 모델은 적절한 함수가 있을 때 자동으로 그 함수를 호출할 수 있습니다. 여기서는 get_current_weather 함수 명세를 통해 사용자가 입력한 위치에 맞는 날씨 정보를 가져올 수 있도록 유도됩니다.

- 만약 모델이 함수 호출을 결정하면, 응답에 function_call이 포함되며, 이때 함수 이름과 그 인자가 함께 반환됩니다. 그런 후에 함수가 호출되고, 결과가 다시 모델에 제공됩니다.

이 과정에서 모델은 단순히 인자를 생성하고, 함수를 호출하는 것은 개발자의 책임입니다. OpenAI의 API는 실제 함수 실행을 수행하지 않으므로, 함수 호출 후 결과를 처리하고 다시 모델에 넘기는 과정을 수동으로 처리하게 됩니다.

In [ ]:
!pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


Open Weather 에서 API key 받아 실시간 날씨 확인

https://home.openweathermap.org/

In [ ]:
import openai
import json
import requests

# OpenAI API 키 설정
from openai import OpenAI
client=OpenAI(api_key='')

# 날쌔ㅣ API 키 설정 (예: OpenWeatherMap)
weather_api_key=''

# 날씨 정보를 가져오는 함수 정의
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response=requests.get(url)
    return response.json()

# 사용자 요청 메시지
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content" : "What is the current weather in New York?"}
]

# GPT-4o-mini 모델 호출
response=client.chat.completions.create(
    model='gpt-4o-mini-2024-07-18',
    messages=messages,
    functions=[{
        "name": "get_current_weather",
        "description": "Get the current weather for a specific location",
        "parameters": {"type": "object",
                       "properties": {"location": {"type": "string","description": "The name of the city to get the weather for"}
            },
            "required": ["location"]
        }
    }],
    function_call="auto"
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다
response_message=response.choices[0].message
messages.append(response_message)

print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"New York"}', name='get_current_weather'), tool_calls=None)


In [ ]:
# (위 셀과 이어서 작성됨)
# 도구 호출 여부 확인
function_call=response_message.function_call
if function_call:
    tool_function_name=function_call.name
    tool_arguments = json.loads(function_call.arguments)

    # 함수 호출 및 결과 처리
    if tool_function_name=='get_current_weather':
        location=tool_arguments['location']
        weather_results=get_current_weather(location)

        # 함수 호출 결과 메세지 추가
        messages.append({
            "role": "function",
            "name": tool_function_name,
            "content": json.dumps(weather_results) # JSON 형식으로 반환
        })

        # 모델 재호출
        model_response_with_function_call=client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    # 도구 호출이 없는 경우 결과 반환
    print(response_message.content)

The current weather in New York is as follows:

- **Temperature:** 21.6°C (feels like 22.17°C)
- **Conditions:** Mist with moderate rain
- **Humidity:** 90%
- **Wind:** Speed of 4.63 m/s from the northeast, with gusts up to 7.72 m/s
- **Visibility:** 4828 meters
- **Rainfall:** 1.65 mm in the last hour
- **Cloud Cover:** 100%

Overall, it's a misty and rainy day in New York.


사용자에게 날씨 확인하고 싶은 도시 이름은 받아서 함수 돌리기

In [ ]:
# 1. 사용자 입력 받아 messages.append로 추가

In [ ]:
import openai
import json
import requests

# OpenAI API 키 설정
from openai import OpenAI
client=OpenAI(api_key='')

# 날쌔ㅣ API 키 설정 (예: OpenWeatherMap)
weather_api_key=''

# 날씨 정보를 가져오는 함수 정의
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response=requests.get(url)
    return response.json()

# 사용자 요청 메시지 초기화
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content" : "Please provide the location to get the current weather information."}
]

# 사용자 입력 받기
user_location=input(str("Enter the location to get the current weather: "))
messages.append({"role": "user", "content": f"What is the current weather in {user_location}"}) # 사용자 입력 받은 내용을 messages 에 추가해주기

# GPT-4o-mini 모델 호출
response=client.chat.completions.create(
    model='gpt-4o-mini-2024-07-18',
    messages=messages,
    functions=[{
        "name": "get_current_weather",
        "description": "Get the current weather for a specific location",
        "parameters": {"type": "object",
                       "properties": {"location": {"type": "string","description": "The name of the city to get the weather for"}
            },
            "required": ["location"]
        }
    }],
    function_call="auto"
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다
response_message=response.choices[0].message
messages.append(response_message)

print(response_message)


# (위 셀과 이어서 작성됨)
# 도구 호출 여부 확인
function_call=response_message.function_call
if function_call:
    tool_function_name=function_call.name
    tool_arguments = json.loads(function_call.arguments)

    # 함수 호출 및 결과 처리
    if tool_function_name=='get_current_weather':
        location=tool_arguments['location']
        weather_results=get_current_weather(location)

        # 함수 호출 결과 메세지 추가
        messages.append({
            "role": "function",
            "name": tool_function_name,
            "content": json.dumps(weather_results) # JSON 형식으로 반환
        })

        # 모델 재호출
        model_response_with_function_call=client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    # 도구 호출이 없는 경우 결과 반환
    print(response_message.content)

Enter the location to get the current weather: New York
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"New York"}', name='get_current_weather'), tool_calls=None)
The current weather in New York is as follows:

- **Condition:** Broken clouds
- **Temperature:** 21.73°C
- **Feels Like:** 22.36°C
- **Humidity:** 92%
- **Wind Speed:** 4.12 m/s (from the northeast)
- **Cloud Cover:** 75%
- **Pressure:** 1008 hPa

If you need more specific details or forecasts, feel free to ask!


In [ ]:
# 2. 사용자 입력을 messages에서 바로 받아 쓰기

In [ ]:
import openai
import json
import requests

# OpenAI API 키 설정
from openai import OpenAI
client=OpenAI(api_key='')

# 날쌔ㅣ API 키 설정 (예: OpenWeatherMap)
weather_api_key=''

# 날씨 정보를 가져오는 함수 정의
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response=requests.get(url)
    return response.json()

# 사용자 입력 받기
user_location=input(str("Enter the location to get the current weather: "))

# 사용자 요청 메시지 초기화
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content" : f"What is the current weather in {user_location}"}
]

# GPT-4o-mini 모델 호출
response=client.chat.completions.create(
    model='gpt-4o-mini-2024-07-18',
    messages=messages,
    functions=[{
        "name": "get_current_weather",
        "description": "Get the current weather for a specific location",
        "parameters": {"type": "object",
                       "properties": {"location": {"type": "string","description": "The name of the city to get the weather for"}
            },
            "required": ["location"]
        }
    }],
    function_call="auto"
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다
response_message=response.choices[0].message
messages.append(response_message)

print(response_message)


# (위 셀과 이어서 작성됨)
# 도구 호출 여부 확인
function_call=response_message.function_call
if function_call:
    tool_function_name=function_call.name
    tool_arguments = json.loads(function_call.arguments)

    # 함수 호출 및 결과 처리
    if tool_function_name=='get_current_weather':
        location=tool_arguments['location']
        weather_results=get_current_weather(location)

        # 함수 호출 결과 메세지 추가
        messages.append({
            "role": "function",
            "name": tool_function_name,
            "content": json.dumps(weather_results) # JSON 형식으로 반환
        })

        # 모델 재호출
        model_response_with_function_call=client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    # 도구 호출이 없는 경우 결과 반환
    print(response_message.content)

Enter the location to get the current weather: Sydney
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Sydney"}', name='get_current_weather'), tool_calls=None)
The current weather in Sydney is as follows:

- **Condition**: Overcast clouds
- **Temperature**: 14.85°C (feels like 14.48°C)
- **Humidity**: 80%
- **Wind Speed**: 0.89 m/s from the east
- **Visibility**: 10,000 meters
- **Cloud Coverage**: 100%

Overall, the weather is quite cloudy and cool.


## How to call functions with model generated arguments

다음 예제에서는 모델에서 생성된 입력을 갖는 함수를 실행하는 방법을 보여주고 이를 사용하여 데이터베이스에 대한 질문에 답할 수 있는 에이전트를 구현합니다. 단순화를 위해 Chinook 샘플 데이터베이스를 사용하겠습니다 .

참고: 모델이 올바른 SQL을 생성하는 데 완벽하게 신뢰할 수 없기 때문에 프로덕션 환경에서 SQL 생성은 위험할 수 있습니다.

이 코드는 OpenAI의 GPT 모델을 사용하여 SQLite 데이터베이스에서 음악 관련 질문에 대한 답변을 SQL 쿼리로 변환한 후, 해당 쿼리를 실행하여 결과를 반환하는 방식으로 function calling을 구현한 예시입니다. 여기서는 사용자가 앨범 관련 질문을 하면, GPT-4가 질문을 SQL 쿼리로 변환하고, SQLite 데이터베이스를 조회하여 결과를 반환합니다.

Specifying a function to execute SQL queries

In [ ]:
import sqlite3
from google.colab import drive

drive.mount('/content/drive')
# sqlite3.connect()를 통해 Google Drive 에 있는 Chinnok.db SQLite 데이터베이스에 연결
conn=sqlite3.connect("/content/drive/MyDrive/KDT_240424/m9_LLM/data/Chinook.db")
print("Opened database successfully")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Opened database successfully


데이터베이스 테이블 및 열 정보 조회

- get_table_names, get_column_names 함수를 통해 데이터베이스의 테이블 및 열 이름을 가져옵니다.
- 이 정보는 나중에 GPT 모델이 SQL 쿼리를 생성할 때 사용할 스키마 정보를 제공하는 데 활용됩니다.

In [ ]:
# 데이터베이스에서 테이블 목록을 추출하는 함수.
# sqlite_master 테이블에서 'type'이 ' table'인 항목들의 이름을 가져와
def get_table_names(conn):
    """Return a list of table names."""
    table_names=[]
    tables=conn.execute("SELECT name FROM sqlite_master WHERE type='table';") # <== execute 괄호 안에 들었는 것은 SQL 쿼리문임
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

# PRAGMA table_info(table_names) 명령을 사용하여 테이블의 스키마 정보를 가져오고, 컬럼 이름을 리스트로 반환
def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names=[]
    columns=conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts=[]
    for table_name in get_table_names(conn):
        columns_names=get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

데이터베이스의 테이블과 열 정보를 문자열 형태로 저장하여, 나중에 SQL 쿼리 작성 시 참조할 수 있도록 함

In [ ]:
database_schema_dict=get_database_info(conn)
database_schema_string="\n".join([
    f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
    for table in database_schema_dict
])

tools라는 리스트에 ask_database라는 함수 명세를 정의합니다.
- 이 함수는 사용자의 질문에 맞는 SQL 쿼리를 받아 데이터베이스에서 정보를 조회하고, 이를 반환하는 기능을 합니다.
- 함수의 매개변수로 query가 있으며, 이는 SQL 쿼리를 텍스트 형태로 전달받아 실행하는 구조입니다.
- 중요한 점은 database_schema_string이 함수 설명에 포함되어 있어, 모델이 데이터베이스 스키마에 맞는 SQL 쿼리를 생성할 수 있도록 도움을 줍니다.

In [ ]:
tools=[
    {
        "type": "function",
        "function": {"name": "ask_database",
        "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
        "parameters": {"type": "object", "properties":{"query":
                                                       {"type": "string",
                                                        "description": f"""SQL query extrating info to answer the user's question.
                                                                           SQL should be written using this database schema:
                                                                           {database_schema_string}
                                                                           The query should be returned in plain text, not in JSON.""",}
                                                       },
                                                        "required": ["query"],
                       },
                    }
    }
]

Executing SQL queries

이제 실제로 데이터베이스에 대한 쿼리를 실행하는 함수를 구현해 보겠습니다.

In [ ]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results=str(conn.execute(query).fetchall())
    except Exception as e:
        results=f"query failed with error: {e}"
    return results

##### Steps to invoke a function call using Chat Completions API:

- 1단계 : 모델이 사용할 도구를 선택하도록 하는 내용으로 모델을 프롬프트합니다. 함수 이름 및 서명과 같은 도구에 대한 설명은 '도구' 목록에 정의되어 API 호출에서 모델에 전달됩니다. 선택한 경우 함수 이름과 매개변수가 응답에 포함됩니다.
- 2단계 : 모델이 함수를 호출하려고 하는지 프로그래밍적으로 확인합니다. 참이면 3단계로 진행합니다.
- 3단계 : 응답에서 함수 이름과 매개변수를 추출하고 매개변수와 함께 함수를 호출합니다. 결과를 메시지에 추가합니다.
- 4단계 : 메시지 목록으로 채팅 완료 API를 호출하여 응답을 가져옵니다.

In [ ]:
# 사용자의 요청 메세지를 정의합니다
messages=[{
    "role": "user", "content": "What is the name of the album with the most tracks?"
}]

# 사용자의 질문에 대한 응답을 생성. tools와 tool_choice 파라미터는 모델이 데이터베이스 쿼리를 인식하고 자동으로 도구를 선택할 수 있도록 돕습니다.
response=client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    tools=tools,
    tool_choice="auto" # 모델이 자동으로 함수 호출을 결정, SQL 쿼리로 변환할 필요가 있다고 판단하면 자동으로 ask_database 함수를 호출하게 됩니다.
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다
response_message=response.choices[0].message
messages.append(response_message)

print(response_message)

# 여기서 tool_calls, function_call 를 지정하지 않았음에도 하기에 포함되어 나오는 이유는 여기서 사용한 모델이 최신 모델이기 때문.
# 위에서 tools라 넣은 이유는 gpt-4o (예전 모델)을 사용했기 때문에 설정해줘야 했던 것.

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_H0utJc8qg0TIigXhgT15ig3l', function=Function(arguments='{"query":"SELECT Album.Title, COUNT(Track.TrackId) AS TrackCount FROM Album JOIN Track ON Album.AlbumId = Track.AlbumId GROUP BY Album.AlbumId ORDER BY TrackCount DESC LIMIT 1;"}', name='ask_database'), type='function')])


In [ ]:
# 모델 응답에서 도구 호출이 포함되어 있는지 확인하고, 도구 호출이 있다면 도구 호출  ID, 함수 이름 및 쿼리 문자열을 추출합니다
tool_calls=response_message.tool_calls
if tool_calls:
    # if true the model will return the name of the tool/ function to call and the argument(s)
    tool_call_id=tool_calls[0].id
    tool_function_name=tool_calls[0].function.name
    tool_query_string=json.loads(tool_calls[0].function.arguments)['query']

    # 도구 호출 함수 이름이 'ask_database'인 경우, ask_database 함수를 호출하여 데이터베이스 쿼리를 실행하고 결과를 messages 리스트에 추가합니다
    if tool_function_name=='ask_database':
        results=ask_database(conn, tool_query_string)

        messages.append({
            "role": "tool",
            "tool_call_id": tool_call_id,
            "name": tool_function_name,
            "content": results
        })

        # 도구 호출 결과가 포함된 messages 리스트를 사용하여 모델을 다시 호출하고 최종 응답을 출력합니다.
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call=client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages,
        ) # get a new response from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
        # 도구 호출 함수 이름이 'ask_database'가 아닌 경우 오류 메세지를 출력하거나, 도구 호출이 없으면 모델의 응답 내용을 바로 출력합니다.
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    # Model did not identify a function to call, result can be returned to the user
    print(response_message.content)

The album with the most tracks is titled "Greatest Hits," which contains a total of 57 tracks.


In [ ]:
import sqlite3
import json
import openai
import sqlite3
from google.colab import drive

drive.mount('/content/drive')

# OpenAI API 키 설정
from openai import OpenAI
client = OpenAI(api_key="")

# 데이터베이스 연결
conn = sqlite3.connect("/content/drive/MyDrive/KDT_240424/m9_LLM/data/Chinook.db")
print("Opened database successfully")

# 함수 정의
def get_table_names(conn):
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

def get_column_names(conn, table_name):
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

# 함수 정의
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

# 사용자의 질문에 대한 응답을 생성하는 함수
messages = [{
    "role":"user",
    "content": "What is the name of the album with the most tracks?"
}]

# 'functions' 정의
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                        SQL query extracting info to answer the user's question.
                        SQL should be written using this database schema:
                        {database_schema_string}
                        The query should be returned in plain text, not in JSON.
                    """
                }
            },
            "required": ["query"]
        }
    }
]

# 모델 응답 요청
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    functions=functions,  # 'tools' 대신 'functions' 사용
    function_call="auto"  # 'tool_choice' 대신 'function_call' 사용
)

# 모델의 응답 메시지와 함수 호출 처리
response_message = response.choices[0].message
messages.append(response_message)


# function_call이 있는지 확인
if response_message.function_call:  # 'function_call' 속성을 직접 확인
    # 함수 호출 정보 추출
    function_name = response_message.function_call.name
    function_arguments = json.loads(response_message.function_call.arguments)

    # 'ask_database' 함수 호출
    if function_name == 'ask_database':
        query = function_arguments['query']
        results = ask_database(conn, query)

        messages.append({
            "role": "function",
            "name": function_name,
            "content": results
        })

        # 모델의 최종 응답 요청
        final_response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages,
        )

        print(final_response.choices[0].message.content)
else:
    # 함수 호출이 없으면 결과 출력
    print(response_message.content)


Mounted at /content/drive
Opened database successfully
The album with the most tracks is titled "Greatest Hits."


## LangChain
- 자연어 처리(NLP)와 생성형 AI 응용 프로그램을 개발하기 위한 프레임워크입니다. 주로 대형 언어 모델(LLMs)과 같은 최신 NLP 기술을 기반으로 하여 다양한 작업을 자동화하거나 개선할 수 있는 도구와 서비스를 제공합니다.
- 이 프레임워크는 특히 언어 모델의 기능을 확장하고 이를 보다 쉽게 사용할 수 있도록 설계되었습니다.
- LangChain의 핵심 목표는 언어 모델을 활용하여 여러 복잡한 작업을 수행할 수 있도록 돕는 것이며, 특히 긴 텍스트, 문서 체인 또는 여러 단계의 워크플로를 필요로 하는 복잡한 응용 프로그램에 유용합니다.

### LangChain의 구성 요소:
- Language Models: 언어 모델 자체를 사용하여 텍스트 생성, 요약, 번역 등의 작업을 수행합니다.
- Chains: 여러 모델 호출을 연결하여 복잡한 작업을 수행하는 논리적 흐름을 정의할 수 있습니다. 예를 들어, 텍스트를 요약한 후 요약된 텍스트에 대한 질의응답을 수행하는 체인을 만들 수 있습니다.
- Agents: 주어진 작업에 따라 다양한 툴을 선택하고 사용할 수 있는 지능형 에이전트입니다. 예를 들어, 정보 검색, API 호출 등을 수행하는 역할을 합니다.
- Memory: 이전의 상호작용 또는 맥락을 기억하는 기능입니다. 이를 통해 대화형 AI나 컨텍스트를 유지해야 하는 애플리케이션을 구현할 수 있습니다.

### 활용 사례:
- 대화형 에이전트: 사용자와의 대화에서 맥락을 유지하며 대답할 수 있는 챗봇 개발.
문서 처리: 긴 문서나 여러 문서의 내용을 요약하거나 분석하는 애플리케이션.
- 지식 탐색: 사용자가 특정 주제에 대해 질문하면, 관련된 정보를 검색하고 이를 바탕으로 대답을 제공하는 시스템.

### 통합:
- LangChain은 다양한 데이터 소스, API 및 언어 모델과 통합될 수 있으며, 이를 통해 다양한 도메인에서 사용될 수 있습니다. 예를 들어, SQL 데이터베이스에 쿼리를 보내고 결과를 요약하거나, 웹에서 정보를 수집한 후 이를 기반으로 질문에 답하는 작업을 수행할 수 있습니다.

이 프레임워크는 특히 연구자, 데이터 사이언티스트, 개발자들이 생성형 AI를 활용하여 복잡한 텍스트 기반 응용 프로그램을 구축하는 데 큰 도움이 됩니다. LangChain은 이러한 작업을 더 쉽게, 더 직관적으로 구현할 수 있게 도와줍니다.

https://www.langchain.com/

https://www.langchain.com/langsmith


LangChain 패키지를 사용하여 RAG 시스템을 구성하는 예제

주어진 질문에 대해 텍스트 파일에 저장된 문서에서 고나련 정보를 검색하고, 이를 기반으로 GPT 모델이 답변을 생성하는 것입니다.

- langchain-openai: LangChain과 OpenAI를 연결하는 패키지.
- faiss-cpu: 벡터 검색을 위한 FAISS 라이브러리의 CPU 버전.
- langchain_community: LangChain의 커뮤니티에서 개발된 추가 도구.
- tiktoken: OpenAI 모델에서 사용하는 토큰화 도구.

In [ ]:
!pip install -q openai
!pip install langchain langchain-openai -q
!pip install faiss-cpu -q
!pip install langchain_community -q
!pip install tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 509.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0

In [ ]:
from openai import OpenAI

## Set the API key and model name
MODEL="gpt-4o-mini-2024-07-18"
client=OpenAI(api_key="")

documents.txt

In [ ]:
# Define the content for both basic and advanced selections
content = """
Document 1:
Polar bears are directly impacted by climate change due to their dependence on sea ice for hunting seals. As global temperatures rise, sea ice is melting earlier and forming later each year, reducing the time polar bears have to hunt. This has led to decreased body condition, lower cub survival rates, and in some cases, increased mortality.

Document 2:
The loss of sea ice habitat is one of the most significant threats to polar bear populations. As the ice retreats, polar bears are forced to travel greater distances to find food, leading to increased energy expenditure. This can result in malnutrition and a decline in reproductive success, further endangering the species.

Document 3:
Climate change is not only affecting polar bears' hunting grounds but also their denning areas. Warmer temperatures and unstable snow conditions are making it more difficult for pregnant females to find suitable denning sites, which is crucial for the birth and survival of their cubs. This adds an additional layer of risk to polar bear populations already struggling due to loss of sea ice.

Document 4:
As their traditional food sources become less accessible, some polar bears have been observed scavenging on human waste or preying on seabirds and their eggs. While this behavior may provide some short-term relief, it does not replace the high-fat diet they obtain from seals, which is essential for their long-term survival in the harsh Arctic environment.

Document 5:
Recent studies suggest that if current trends in greenhouse gas emissions continue, polar bears could face extinction within this century. Conservation efforts are focusing on reducing global emissions and protecting critical polar bear habitats, but these efforts may not be sufficient if the climate continues to warm at the current rate.
"""

# Write the content to a file
with open('documents.txt', 'w') as file:
    file.write(content)

In [ ]:
!ls

documents.txt  sample_data


In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

- OpenAIEmbeddings: 문서를 벡터로 임베딩하는 클래스입니다. 각 문서를 숫자 벡터로 변환하여 유사성 검색을 할 수 있게 만듭니다.
- ChatOpenAI: OpenAI의 GPT 모델을 사용하는 클래스입니다. model을 지정하여 특정 GPT 모델을 사용할 수 있으며, fine-tuned된 모델도 지원합니다.
- FAISS (Facebook AI Similarity Search): 벡터를 빠르게 검색하기 위한 라이브러리입니다. Facebook에서 개발한 AI 유사성 검색 라이브러리로, 임베딩된 문서들 중에서 가장 관련성이 높은 문서를 빠르게 검색할 수 있습니다.
- TextLoader: 문서를 텍스트 형식으로 불러오는 클래스입니다. 이 클래스는 주어진 파일에서 텍스트 데이터를 로드할 때 사용됩니다.
- RetrievalQA: 검색된 문서와 GPT 모델을 결합하여 질의응답 작업을 수행하는 체인입니다.

In [ ]:
# OpenAI API 키 설정
import os
os.environ['OPENAI_API_KEY']='' # OPEN AI APi 키 입력

# 1. 문서 로드
# 텍스트 파일을 사용해 문자를 로드합니다
loader=TextLoader('documents.txt')
documents=loader.load()

이 코드는 문서를 로드하는 부분입니다.

- **TextLoader**를 사용하여 documents.txt 파일을 불러옵니다. 이 파일은 시스템에서 검색할 수 있는 문서들이 포함된 파일입니다.
- **loader.load()**를 호출하여 문서를 읽어들이고, 이를 documents라는 변수에 저장합니다. 이 변수는 나중에 벡터화(임베딩)할 때 사용됩니다.

In [ ]:
# 2. 문서 임베딩 및 인덱싱
# 문서를 벡터로 변환한 후, FAISS 벡터 저장소에 자정합니다
embeddings = OpenAIEmbeddings()
vector_store=FAISS.from_documents(documents, embeddings)

이 부분은 문서를 벡터로 변환하고 이를 인덱싱하는 단계입니다.

- OpenAIEmbeddings: 문서를 숫자 벡터로 변환하는데 사용됩니다. 임베딩은 텍스트의 의미를 벡터 공간에서 표현하는 방법으로, 유사성 검색에 중요한 역할을 합니다.
- FAISS.from_documents: documents 변수를 임베딩한 후, FAISS 인덱스를 생성합니다. 이 인덱스는 문서를 벡터로 변환한 후 빠르게 유사성을 계산하고 관련 문서를 검색하는 데 사용됩니다.

In [ ]:
# 3. Fine-tuned GPT 모델을 사용한 LLM 생성
# Fine-tuned된 GPT 모델을 사용할 수 있도록 설정합니다.
llm=ChatOpenAI(model='gpt-4o-mini-2024-07-18', temperature=0.3)

이 부분은 GPT 모델을 설정하는 부분입니다.

- ChatOpenAI: OpenAI의 GPT 모델을 사용합니다. 여기서 model 파라미터로 Fine-tuned된 GPT 모델을 지정하고 있습니다. "gpt-4o-mini-2024-07-18"은 Fine-tuning된 GPT 모델의 ID입니다.
- temperature: 텍스트 생성을 제어하는 파라미터로, 값이 낮을수록 더 결정적인(deterministic) 결과를 생성하고, 값이 높을수록 생성되는 텍스트의 창의성(랜덤성)이 높아집니다.

In [ ]:
# 4. 문서 검색 개수 조정 (최대 5개의 문서 검색)
# 사용자가 입력한 쿼리와 가장 유사한 5개의 결과를 검색하여 반환하겠다는 의미
retriever = vector_store.as_retriever(search_kwargs={'k':5})

이 부분에서는 검색할 문서의 개수를 설정합니다.

- retriever: vector_store에서 as_retriever 메서드를 사용하여 문서 검색을 위한 리트리버(retriever)를 생성합니다.
- k 값: search_kwargs에서 k를 5로 설정하여, 사용자가 질문을 할 때 최대 5개의 문서가 검색되도록 설정합니다. 즉, 질문과 관련된 상위 5개의 문서를 검색하여 답변에 사용할 수 있도록 합니다.

In [ ]:
# 5. RAG 구현
# RetrievalQA 체인을 사용해 검색된 문서를 기반으로 LLM을 통해 답변을 생성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, # Fine-tuned된 GPT 모델
    chain_type="stuff", # 'stuff' 체인을 사용하여 검색된 문서들을 단일 입력으로 결합하여 처리.
    retriever=retriever, # 검색할 문서의 수를 5개로 설정한 retriever
    return_source_documents=True # 검색된 문서를 포함하여 반환할지 여부를 지정
)

이 부분은 Retrieval-Augmented Generation (RAG) 체인을 구현하는 부분입니다.

- RetrievalQA: RetrievalQA 체인은 검색된 문서를 GPT 모델과 결합하여 질문에 대한 답변을 생성합니다.
- from_chain_type: 여기서는 "stuff" 체인 유형을 사용합니다. 이는 검색된 문서들을 모두 결합하여 하나의 텍스트로 처리하는 방식입니다.
  다른 체인 방식으로는 map-reduce나 refine 등의 방법이 있습니다.
  - Map-reduce: 문서를 나눠서 처리하고, 나중에 결합하는 방식.
  - Refine: 문서를 한 번에 하나씩 처리하고, 그 결과를 점진적으로 개선하는 방식.
- retriever: 이 retriever는 벡터 스토어(예: FAISS)에서 관련 문서를 검색한 후, 해당 문서들을 LLM에게 제공하여 답변을 생성하도록 돕습니다.
- return_source_documents=True: 검색된 문서를 응답에 포함하여 반환할지 여부를 지정합니다. True로 설정하면, 검색된 문서들을 답변과 함께 반환합니다. 이렇게 하면 사용자가 생성된 답변 외에도 검색된 문서들을 검토할 수 있습니다.

In [ ]:
# 6. 질문에 대한 답변 생성
query='What is the impact of climate change on polar bears?'
result=qa_chain.invoke({'query': query}) # invoke 메서드를 사용하여 query 전달

# 7. 결과 출력
print(result['result'])

Climate change has a significant impact on polar bears primarily due to their dependence on sea ice for hunting seals. As global temperatures rise, sea ice is melting earlier and forming later each year, which reduces the time polar bears have to hunt. This has led to decreased body condition, lower cub survival rates, and increased mortality in some cases.

The loss of sea ice habitat forces polar bears to travel greater distances to find food, resulting in increased energy expenditure, malnutrition, and a decline in reproductive success. Additionally, warmer temperatures and unstable snow conditions make it more difficult for pregnant females to find suitable denning sites, which is crucial for the birth and survival of their cubs.

As traditional food sources become less accessible, some polar bears have started scavenging on human waste or preying on seabirds and their eggs, but this behavior does not provide the high-fat diet they need for long-term survival. If current trends in 